### Análise da base de dados 'Beta' utilizando algoritmos de ML

Neste notebook será analisado o 'Beta Dataset' utilizando algoritmos de ML para realizar (1) extração de característocas; (2) seleção de características e (3) classificação dos dados.

#### Pontos importantes
- Frequências estimuladas (tota de 40, com a diferença de 0.2 Hz uma da outra): 8.9; 8.2, ...., 15.6, 15.8;
- Taxa de amostragem: 250Hz

### Manipulação dos dados

In [30]:
import numpy as np
import mne
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_selection import RFECV

In [31]:
data = np.load("../../datasets/beta/data.npy")
print(data.shape)

(160, 64, 750)


In [32]:
# criação de um objeto "info"
n_channels = 64
sfreq = 250
ch_names = ["FP1", "FPZ", "FP2", "AF3", "AF4", "F7", "F5", "F3", "F1", "FZ", "F2", "F4", "F6", "F8", "FT7", "FC5", "FC3", "FC1", "FCZ", "FC2", "FC4", "FC6", "FT8", "T7", "C5", "C3", "C1", "CZ", "C2", "C4", "C6", "T8", "M1", "TP7", "CP5", "CP3", "CP1", "CPZ", "CP2", "CP4", "CP6", "TP8", "M2", "P7", "P5", "P3", "P1", "PZ", "P2", "P4", "P6", "P8", "PO7", "PO5", "PO3", "POZ", "PO4", "PO6", "PO8", "CB1", "O1", "OZ", "O2", "CB2"]
ch_types = ['eeg'] * n_channels
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)

In [33]:
labels = np.load("../../datasets/beta/labels.npy")
print(labels, labels.shape)

[ 8.6  8.8  9.   9.2  9.4  9.6  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2
 11.4 11.6 11.8 12.  12.2 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.
 14.2 14.4 14.6 14.8 15.  15.2 15.4 15.6 15.8  8.   8.2  8.4  8.6  8.8
  9.   9.2  9.4  9.6  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2 11.4 11.6
 11.8 12.  12.2 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.  14.2 14.4
 14.6 14.8 15.  15.2 15.4 15.6 15.8  8.   8.2  8.4  8.6  8.8  9.   9.2
  9.4  9.6  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2 11.4 11.6 11.8 12.
 12.2 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.  14.2 14.4 14.6 14.8
 15.  15.2 15.4 15.6 15.8  8.   8.2  8.4  8.6  8.8  9.   9.2  9.4  9.6
  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2 11.4 11.6 11.8 12.  12.2 12.4
 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.  14.2 14.4 14.6 14.8 15.  15.2
 15.4 15.6 15.8  8.   8.2  8.4] (160,)


In [34]:
# método para transformar labels categóricos
data_correct = data
le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(160, dtype=int),
    le.fit_transform(labels))
)

event_dict = {str(value): index  for index, value in enumerate(sorted(set(labels)))}
mne_data = mne.EpochsArray(data_correct, info, events, event_id=event_dict)
filtered_mne_data = mne_data.filter(6, 18)
filtered_mne_data

Not setting metadata
160 matching events found


No baseline correction applied
0 projection items activated
Setting up band-pass filter from 6 - 18 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 6.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Upper passband edge: 18.00 Hz
- Upper transition bandwidth: 4.50 Hz (-6 dB cutoff frequency: 20.25 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 3041 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 3527 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 4607 tasks      | elapsed:    0.8s
[Parallel(n_job

Number of events,160
Events,10.0: 410.2: 410.4: 410.6: 410.8: 411.0: 411.2: 411.4: 411.6: 411.8: 412.0: 412.2: 412.4: 412.600000000000001: 412.8: 413.0: 413.200000000000001: 413.4: 413.600000000000001: 413.8: 414.0: 414.200000000000001: 414.4: 414.600000000000001: 414.8: 415.0: 415.200000000000001: 415.4: 415.600000000000001: 415.8: 48.0: 48.2: 48.4: 48.6: 48.799999999999999: 49.0: 49.2: 49.4: 49.6: 49.8: 4
Time range,0.000 – 2.996 s
Baseline,off


In [35]:
# SNR

shape = data.shape
# Estimando o ruído de fundo

# intervalos de tempo sem estímulo (0 a 0,5 segundos e 2,5 a 3 segundos)
base_start = 0
base_end = 125
rest_start = 625
rest_end = 750

# armazena uma lista com as médias de potência para cada canal
noise_power = []
# consideramos a primeira amostra (1º target, 1º trial)
for channel_data in data[0, :, :]:
    fft_result = np.fft.fft(channel_data)
    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    # média da potência nos intervalos de tempo sem estímulo
    base_power = np.mean(psd[base_start:base_end])
    rest_power = np.mean(psd[rest_start:rest_end])
    # média das duas médias de potência obtidas anteriormente
    mean_noise_power = (base_power + rest_power) / 2
    noise_power.append(mean_noise_power)
#média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
estimated_background_noise

29962.711288130937

In [36]:
sr = 250

# Frequências alvo
target_frequencies = np.arange(8, 16, 0.2)
target_amplitudes = []

for channel_data in mne_data.get_data():
    target = []
    for eletrodo in channel_data:
        fft_result = np.fft.fft(eletrodo)
        psd = np.abs(fft_result) ** 2
        frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)
        target_amplitudes_trial = []
        for target_frequency in target_frequencies:
            # Encontrando o índice da frequência alvo no espectro de frequência
            index = np.argmin(np.abs(frequencies - target_frequency))
            # Amplitude na frequência alvo
            amplitude = np.sqrt(psd[index])
            target_amplitudes_trial.append(amplitude)
        target.append(target_amplitudes_trial)
    
    target_amplitudes.append(target)

target_amplitudes = np.array(target_amplitudes)
target_amplitudes.shape

(160, 64, 40)

In [37]:
target_amplitudes_adjusted = np.abs(target_amplitudes - mean_noise_power)

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / mean_noise_power)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)


data = np.array([narrow_band_SNR, wide_band_SNR])
data = data.swapaxes(0, 1)
data = data.swapaxes(1, 2)
data = data.reshape(data.shape[0], data.shape[1], data.shape[2] * data.shape[3])
print(data.shape)

(160, 64, 80)


### Determinação das características

#### 1. Seleção de característica manual

In [38]:
channels_to_keep = ['PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'POZ', 'O1', 'OZ', 'O2']

# Crie uma máscara booleana para os canais a serem mantidos
mask = np.isin(ch_names, channels_to_keep)

# Use a máscara para selecionar os canais desejados na matriz data
data_filtered = data[:, mask, :]

print("Dados filtrados com 9 eletrodos", data_filtered.shape)

selected_ch = data_filtered.reshape(data_filtered.shape[0], data_filtered.shape[1] * data_filtered.shape[2])
print(selected_ch.shape)

Dados filtrados com 9 eletrodos (160, 9, 80)
(160, 720)


#### 2. Vetor de características sem seleção

In [39]:
print(data.shape)
# reorganizando os dados
data_reshape = data.reshape(data.shape[0], data.shape[1] * data.shape[2])
print(data_reshape.shape)

(160, 64, 80)
(160, 5120)


### Treinamento com SVM

#### Seleção de característica manual com SVM

In [40]:
# Normalizando os dados
X = StandardScaler().fit_transform(selected_ch)
y = LabelEncoder().fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
svm_model = SVC(kernel='linear', probability=True, C=1)
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy:.2%}")


Acurácia: 35.42%


#### Seleção de característica manual com validação cruzada

In [42]:
from sklearn.model_selection import StratifiedKFold

# Criando um objeto StandardScaler
scaler = StandardScaler()

# Normalizando os dados
X = scaler.fit_transform(selected_ch)
y = LabelEncoder().fit_transform(labels)

# Dividindo os dados normalizados em 2 conjuntos (folds) para validação cruzada
cv = StratifiedKFold(n_splits=2)

# Lista para armazenar as acurácias para cada fold
acc_scores = []

# Loop pelos folds
for train_idx, test_idx in cv.split(X, y):
    # Divida os dados normalizados em conjunto de treinamento e conjunto de teste para este fold
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Treine o modelo SVM no conjunto de treinamento
    svm_model.fit(X_train, y_train)

    # Faça previsões no conjunto de teste
    y_pred = svm_model.predict(X_test)

    # Calcule a acurácia do modelo para este fold
    fold_accuracy = accuracy_score(y_test, y_pred)

    # Armazene a acurácia deste fold na lista
    acc_scores.append(fold_accuracy)

mean_acc = np.mean(acc_scores)

# Imprima a média e o desvio padrão das acurácias
print(f"Acurácia média: {mean_acc:.2%}")

Acurácia média: 55.00%


#### 2. Vetor de características sem seleção